# Notebook to check L-BFGS

In [1]:
import numpy as np
import scipy.linalg as la

In [2]:
B_0 = np.eye(2)
S_k = np.array([[4, 0, 2, 1],[3, 2, -1, 0]])
Y_k = np.array([[-4, 5, 6, 9],[7, 5, 8, -1]])

In [3]:
sTy = np.dot(S_k.T, Y_k)
sTy

array([[  5,  35,  48,  33],
       [ 14,  10,  16,  -2],
       [-15,   5,   4,  19],
       [ -4,   5,   6,   9]])

In [4]:
L_k = np.tril(sTy, -1)
L_k

array([[  0,   0,   0,   0],
       [ 14,   0,   0,   0],
       [-15,   5,   0,   0],
       [ -4,   5,   6,   0]])

In [5]:
D_k = np.diag(np.diag(sTy))
D_k

array([[ 5,  0,  0,  0],
       [ 0, 10,  0,  0],
       [ 0,  0,  4,  0],
       [ 0,  0,  0,  9]])

In [6]:
# This is how we'd calculate delta_k for L-BFGS
Y_k_mius_1 = Y_k[:,-1]
delta_k = np.dot(Y_k_mius_1, Y_k_mius_1) / np.dot(S_k[:,-1], Y_k_mius_1)
delta_k

9.11111111111111

In [7]:
# But for now we'll set it to 1.
delta_k = 1.

In [8]:
M = np.block([
    [delta_k * np.dot(S_k.T, S_k), L_k],
    [L_k.T, -D_k]
])
M

array([[ 25.,   6.,   5.,   4.,   0.,   0.,   0.,   0.],
       [  6.,   4.,  -2.,   0.,  14.,   0.,   0.,   0.],
       [  5.,  -2.,   5.,   2., -15.,   5.,   0.,   0.],
       [  4.,   0.,   2.,   1.,  -4.,   5.,   6.,   0.],
       [  0.,  14., -15.,  -4.,  -5.,   0.,   0.,   0.],
       [  0.,   0.,   5.,   5.,   0., -10.,   0.,   0.],
       [  0.,   0.,   0.,   6.,   0.,   0.,  -4.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,  -9.]])

In [9]:
dSY = np.block([
    [delta_k * S_k, Y_k]
])
dSY

array([[ 4.,  0.,  2.,  1., -4.,  5.,  6.,  9.],
       [ 3.,  2., -1.,  0.,  7.,  5.,  8., -1.]])

In [10]:
X = la.solve(M, dSY.T)
X

array([[ 0.00000000e+00, -1.48392415e-01],
       [ 0.00000000e+00,  5.15663644e-01],
       [ 0.00000000e+00, -3.44600165e-01],
       [ 1.00000000e+00,  1.33470734e+00],
       [ 1.12692671e-18,  9.89282770e-03],
       [ 2.90598360e-18, -4.94641385e-03],
       [-1.78906766e-18,  2.06100577e-03],
       [-1.00000000e+00,  1.11111111e-01]])

In [11]:
B_k = delta_k * np.eye(2) - np.dot(dSY, X)
B_k

array([[ 9.        , -1.        ],
       [-1.        ,  0.11935513]])

In [12]:
eigenval, eigenvec = la.eig(B_k)
eigenval

array([9.11121174e+00+0.j, 8.14339628e-03+0.j])

In [13]:
eigenvec

array([[ 0.99387275,  0.11053032],
       [-0.11053032,  0.99387275]])

## How to keep only $m$ vectors?

In [14]:
from collections import deque

In [15]:
m, n = 3, 4             # We'll keep m vectors of n elements each
S_k = deque(maxlen=m)
Y_k = deque(maxlen=m)

In [16]:
s_k = np.random.randint(low=-9, high=9, size=n)
y_k = np.random.randint(low=-9, high=9, size=n)
s_k

array([ 7, -8,  2, -1])

In [17]:
for i in range(5):
    s_k = np.random.randint(low=-9, high=9, size=n)
    y_k = np.random.randint(low=-9, high=9, size=n)

    S_k.append(s_k)
    Y_k.append(s_k)

    print(np.array(S_k).T, "\n")

[[ 5]
 [-6]
 [-2]
 [-3]] 

[[ 5 -7]
 [-6  2]
 [-2 -4]
 [-3  0]] 

[[ 5 -7 -9]
 [-6  2 -9]
 [-2 -4  4]
 [-3  0  3]] 

[[-7 -9  8]
 [ 2 -9 -6]
 [-4  4 -5]
 [ 0  3 -9]] 

[[-9  8  2]
 [-9 -6  7]
 [ 4 -5 -9]
 [ 3 -9  3]] 



### Another alternative

But I'd have to reorder, don't like this solution

In [18]:
matrix = np.zeros((n, m))  # Preallocate
index = 0

def update_matrix(matrix, new_vector, index):
    matrix[:, index] = new_vector  # Overwrite the next position
    index = (index + 1) % m  # Circular indexing
    return matrix, index

for i in range(5):
    new_vector = np.random.randint(low=-9, high=9, size=n)
    matrix, index = update_matrix(matrix, new_vector, index)
    print(matrix, "\n")

[[ 8.  0.  0.]
 [ 5.  0.  0.]
 [ 1.  0.  0.]
 [-2.  0.  0.]] 

[[ 8. -8.  0.]
 [ 5. -9.  0.]
 [ 1. -1.  0.]
 [-2.  3.  0.]] 

[[ 8. -8. -2.]
 [ 5. -9.  4.]
 [ 1. -1.  6.]
 [-2.  3.  1.]] 

[[-2. -8. -2.]
 [ 4. -9.  4.]
 [ 3. -1.  6.]
 [-6.  3.  1.]] 

[[-2.  3. -2.]
 [ 4. -2.  4.]
 [ 3. -1.  6.]
 [-6. -9.  1.]] 

